In [1]:
import numpy as np
#from sklearn.metrics import jaccard_similarity_score

In [2]:
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("data/comoda_data.csv")

In [5]:
df = df.replace(-1.0, np.nan)
df = df.dropna(how="any")

In [ ]:
def jaccard_similarity_score(x, y):
    instersection = np.array(x) & np.array(y)
    union = np.logical_or(x, y).astype('int')
    numerator = np.dot(weights, intersection)
    denom = np.dot(weights, union)
    similarity = numerator/denom
    return similarity

## Code DCW Here

In [6]:
## CONSTANTS ASSUMPTION
sigma = 1
threshold = 0.1
columns = df.columns.tolist()
columns.remove("rating")
columns.append("rating")
df = df[columns]
X = df[columns[2:-1]]
y = df[columns[-1:]]
df = df.reset_index(drop=True)
#df

In [7]:
class neighbors:
    def __init__(self, X, y):
        self.model = KNeighborsClassifier(n_neighbors=10)
        self.model.fit(X,y)
    def get_neighbors_data(self, user, itemID):
        #all_neighbors_indices = self.model.kneighbors(user[2:-1])[1][0]
        #print(all_neighbors_indices)
        #neighbors_data = df.iloc[all_neighbors_indices]
        neighbors_data = df[(df.itemID == itemID)]
        #print(neighbors_data)
        self.good_neighbors = pd.DataFrame()
        #print(neighbors_data.shape)
        threshold = 0.3
        for index, row in neighbors_data.iterrows():
            similarity = jaccard_similarity_score(row[2:-1], user[2:-1])
            #print(similarity)
            if similarity > threshold:
                self.good_neighbors=self.good_neighbors.append(row, ignore_index=True)
                #print(row)
                #print(good_neighbors)
                #print("Inside Threshold")
            #print(self.good_neighbors)
        #print(self.good_neighbors)
        return self.good_neighbors

In [12]:
class neighborhood_contribution:
    def __init__(self):
        pass
    def neighbor_rating(self, neighbor, itemID, sigma, threshold):
        self.ratings_sum=0
        self.similarity_sum=0
        #print(type(user))
        #print(user)
        #print(neighbor[2:10])
        #print(df.itemID)
        ratings = df[(df.userID == neighbor.userID) & (df.itemID == itemID)]
        #print(ratings.shape)
        for index, user_rating in ratings.iterrows():
            similarity = jaccard_similarity_score(neighbor[2:-1],user_rating[2:-1])
            #print(similarity)
            if similarity > threshold:
                self.ratings_sum += user_rating.rating * similarity
                self.similarity_sum += similarity
                #print(self.similarity_sum)
            #print(self.ratings_sum)
            #print(neighbor[-10:],user_rating[-10:])
        #print("Rating sum")
        #print(self.ratings_sum)
        #print("Similarity Sum")
        #print(self.similarity_sum)
        try:
            rating = self.ratings_sum / self.similarity_sum
            #rating = ratings.rating.mean()
        except:
            rating = 0
        #print(rating)
        return rating
    def neighbor_average(self,neighbor,sigma,threshold):
        ratings_of_neighbor = df[df.userID == neighbor.userID]#.rating.mean()
        rating_sum=0;
        count=0;
        for index, row in ratings_of_neighbor.iterrows():
            similarity = jaccard_similarity_score(row[2:-1], neighbor[2:-1])
            if similarity > threshold:
                rating_sum+=row.rating
                count+=1
        average_rating_in_given_context = rating_sum/count
        #print(average_rating_in_given_context)
        return average_rating_in_given_context
    def baseline_rating(self, itemID):
        average_item_rating = df[df.itemID == itemID].rating.mean()
        return average_item_rating

In [10]:
def numerator(neighborhood_contribution_rating, similarity_of_neighbors):
    numerator_sum=0
    #print(neighborhood_contribution_rating)
    #print(similarity_of_neighbors)
    for neighbor_rating, similarity in zip(neighborhood_contribution_rating,similarity_of_neighbors):
        numerator_sum+= neighbor_rating*similarity
    return numerator_sum

In [11]:
def predict_rating(user,itemID):
    similarity_function = similarity_class()
    user_neighbors = neighbors(X,y)
    user_neighborhood_contribution = neighborhood_contribution()
    neighbors_data = user_neighbors.get_neighbors_data(user,itemID)
    #print(neighbors_data.shape)
    #print(neighbors_data)
    #similarity_of_neighbors = [similarity_function.jaccard_similarity(neighbor[2:-1],user[2:-1]) for neighbor in neighbors_data]
    similarity_of_neighbors = []
    neighborhood_contribution_rating = []
    #print(neighbors_data.shape)
    for index, neighbor in neighbors_data.iterrows():
        similarity_of_neighbors.append(similarity_function.jaccard_similarity(neighbor[2:-1],user[2:-1]))
        neighborhood_contribution_rating.append(user_neighborhood_contribution.neighbor_rating(neighbor,itemID,1,0.1) - user_neighborhood_contribution.neighbor_average(neighbor,1,0.1))
#     neighborhood_contribution_rating = [user_neighborhood_contribution.neighbor_rating(neighbor,item) - user_neighborhood_contribution.neighbor_average(neighbor)
#                                         for neighbor in neighbors_data]
    numerator_value = numerator(neighborhood_contribution_rating, similarity_of_neighbors)
    baseline_rating = user_neighborhood_contribution.baseline_rating(itemID)
    #print(numerator_value)
    #print(similarity_of_neighbors)
    #print(baseline_rating)
    final_rating = baseline_rating + numerator_value/ sum(similarity_of_neighbors)
    return final_rating

In [12]:
# if __name__ == "main":
#     userId = 20
#     iteId = 30
#     rating = predict_rating(20,30)
#     print(rating)

In [13]:
predict_rating(df.iloc[37], 47)
#df.iloc[5]"

5.0479068684211841

In [14]:
#df[df.itemID == 47]

In [15]:
#df.iloc[37]

In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
#y_true = [df.iloc[i].rating for i in range(700,800)]

In [18]:
#y_pred = [predict_rating(df.iloc[i], df.iloc[i].itemID) for i in range(700,800)]

In [19]:
#y_pred

In [20]:
#mean_squared_error(y_true, y_pred)

## Clean Data

In [21]:
len(df.rating.unique())

5

In [22]:
columns = df.columns

In [23]:
columns = columns[2:-1]

In [24]:
columns

Index(['age', 'sex', 'city', 'country', 'time', 'daytype', 'season',
       'location', 'weather', 'social', 'endEmo', 'dominantEmo', 'mood',
       'physical', 'decision', 'interaction', 'director', 'movieCountry',
       'movieLanguage', 'movieYear', 'genre1', 'genre2', 'genre3', 'actor1',
       'actor2', 'actor3', 'budget'],
      dtype='object')

In [25]:
#from collections import defaultdict
# size_of_feature = defaultdict(dict)
# for column in columns:
#     size_of_feature[column]['length'] = len(df[column].unique())
#     size_of_feature[column]['names'] = df[column].unique()
    

In [26]:
#size_of_feature

In [27]:
#size_of_feature['actor1']

In [28]:
df.actor1.max()

1994.0

In [29]:
df.actor1.describe()

count    1114.000000
mean     1012.370736
std       568.663979
min         4.000000
25%       531.000000
50%       969.000000
75%      1518.250000
max      1994.000000
Name: actor1, dtype: float64

In [30]:
# #from collections import defaultdict
# feature_dict = defaultdict(dict)
# for k,v in size_of_feature.items():
#     for i,j in v.items():
#         for name in v['names']:
#             feature_dict[k][name] = sum(df[k] == name)
#             #print(k,name)

In [31]:
#feature_dict

In [32]:
#df[df.columns[2:]].corr(method='kendall', min_periods=1)

In [33]:
df.columns

Index(['userID', 'itemID', 'age', 'sex', 'city', 'country', 'time', 'daytype',
       'season', 'location', 'weather', 'social', 'endEmo', 'dominantEmo',
       'mood', 'physical', 'decision', 'interaction', 'director',
       'movieCountry', 'movieLanguage', 'movieYear', 'genre1', 'genre2',
       'genre3', 'actor1', 'actor2', 'actor3', 'budget', 'rating'],
      dtype='object')

In [34]:
df_new = df.drop(['city','physical','director','movieCountry','movieLanguage','movieYear','genre1', 'genre2',
       'genre3', 'actor1', 'actor2', 'actor3','budget'], axis=1,)

In [35]:
#df_new.describe()

In [36]:
from collections import defaultdict
new_size_of_feature = defaultdict(dict)
for column in df_new.columns:
    new_size_of_feature[column]['length'] = len(df_new[column].unique())
    new_size_of_feature[column]['names'] = df_new[column].unique()

In [37]:
sum_of_features = 0
for k,v in new_size_of_feature.items():
    sum_of_features += v['length']

In [38]:
sum_of_features

752

In [39]:
df_new.columns

Index(['userID', 'itemID', 'age', 'sex', 'country', 'time', 'daytype',
       'season', 'location', 'weather', 'social', 'endEmo', 'dominantEmo',
       'mood', 'decision', 'interaction', 'rating'],
      dtype='object')

In [40]:
df_age = pd.get_dummies(df.age, prefix='age')

In [41]:
#df_age.head()
df_new.head()

,userID,itemID,age,sex,country,time,daytype,season,location,weather,social,endEmo,dominantEmo,mood,decision,interaction,rating
0,21,5,28.0,1,3,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,3
1,21,6,28.0,1,3,4.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,4
2,22,13,28.0,1,2,3.0,2.0,3.0,2.0,2.0,3.0,2.0,3.0,1.0,1.0,1.0,4
3,21,7,28.0,1,3,4.0,2.0,2.0,1.0,1.0,2.0,1.0,7.0,1.0,2.0,1.0,3
4,21,15,28.0,1,3,3.0,1.0,2.0,1.0,5.0,1.0,7.0,5.0,1.0,2.0,2.0,3


In [42]:
for column in df_new.columns[2:-1]:
    df_col = pd.get_dummies(df_new[column], prefix=column)
    df_new = pd.concat([df_new, df_col], axis=1)

In [43]:
df_new = df_new.drop(['age', 'sex', 'country', 'time', 'daytype',
       'season', 'location', 'weather', 'social', 'endEmo', 'dominantEmo',
       'mood', 'decision', 'interaction'], axis=1)

In [44]:
df_new.shape

(1114, 77)

In [45]:
df_new.head()

,userID,itemID,rating,age_15.0,age_17.0,age_18.0,age_19.0,age_20.0,age_21.0,age_22.0,...,dominantEmo_5.0,dominantEmo_6.0,dominantEmo_7.0,mood_1.0,mood_2.0,mood_3.0,decision_1.0,decision_2.0,interaction_1.0,interaction_2.0
0,21,5,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,21,6,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,22,13,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
3,21,7,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,21,15,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


In [46]:
columns = df_new.columns.tolist()
columns.remove("rating")
columns.append("rating")
df_new = df_new[columns]

In [47]:
df_new.head()

,userID,itemID,age_15.0,age_17.0,age_18.0,age_19.0,age_20.0,age_21.0,age_22.0,age_23.0,...,dominantEmo_6.0,dominantEmo_7.0,mood_1.0,mood_2.0,mood_3.0,decision_1.0,decision_2.0,interaction_1.0,interaction_2.0,rating
0,21,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,3
1,21,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,4
2,22,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,4
3,21,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3
4,21,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,3


In [48]:
#df_new.iloc[0][2:-1].tolist()

In [49]:

#user_rating = df_new.rating.tolist()[:100]

In [50]:
entriesToRemove = ('userID','itemID','city','physical','director','movieCountry','movieLanguage','movieYear','genre1', 'genre2',
       'genre3', 'actor1', 'actor2', 'actor3','budget','rating')

In [51]:
for k in entriesToRemove:
    new_size_of_feature.pop(k, None)

In [52]:
for k,v in new_size_of_feature.items():
    print(v['length'])

21
2
7
5
3
4
3
7
3
4
2
4
2
7


In [53]:
new_size_of_feature

defaultdict(dict,
            {'age': {'length': 21,
              'names': array([ 28.,  30.,  33.,  25.,  26.,  27.,  24.,  34.,  63.,  29.,  21.,
                      22.,  15.,  18.,  17.,  23.,  31.,  20.,  35.,  19.,  36.])},
             'country': {'length': 4,
              'names': array([3, 2, 5, 4], dtype=int64)},
             'daytype': {'length': 3, 'names': array([ 2.,  1.,  3.])},
             'decision': {'length': 2, 'names': array([ 2.,  1.])},
             'dominantEmo': {'length': 7,
              'names': array([ 2.,  3.,  7.,  5.,  4.,  6.,  1.])},
             'endEmo': {'length': 7,
              'names': array([ 2.,  1.,  7.,  4.,  5.,  6.,  3.])},
             'interaction': {'length': 2, 'names': array([ 2.,  1.])},
             'location': {'length': 3, 'names': array([ 1.,  2.,  3.])},
             'mood': {'length': 3, 'names': array([ 2.,  1.,  3.])},
             'season': {'length': 4, 'names': array([ 2.,  3.,  1.,  4.])},
             'sex': {'lengt

In [54]:
df.columns

Index(['userID', 'itemID', 'age', 'sex', 'city', 'country', 'time', 'daytype',
       'season', 'location', 'weather', 'social', 'endEmo', 'dominantEmo',
       'mood', 'physical', 'decision', 'interaction', 'director',
       'movieCountry', 'movieLanguage', 'movieYear', 'genre1', 'genre2',
       'genre3', 'actor1', 'actor2', 'actor3', 'budget', 'rating'],
      dtype='object')

In [55]:
index_limit=[]
for x in df.columns:
    index_limit.append(new_size_of_feature[x])

In [56]:
index_limit = index_limit[2:-12]

In [57]:
index_limit

[{'length': 21,
  'names': array([ 28.,  30.,  33.,  25.,  26.,  27.,  24.,  34.,  63.,  29.,  21.,
          22.,  15.,  18.,  17.,  23.,  31.,  20.,  35.,  19.,  36.])},
 {'length': 2, 'names': array([1, 2], dtype=int64)},
 {},
 {'length': 4, 'names': array([3, 2, 5, 4], dtype=int64)},
 {'length': 4, 'names': array([ 2.,  4.,  3.,  1.])},
 {'length': 3, 'names': array([ 2.,  1.,  3.])},
 {'length': 4, 'names': array([ 2.,  3.,  1.,  4.])},
 {'length': 3, 'names': array([ 1.,  2.,  3.])},
 {'length': 5, 'names': array([ 1.,  2.,  5.,  3.,  4.])},
 {'length': 7, 'names': array([ 2.,  3.,  1.,  5.,  7.,  6.,  4.])},
 {'length': 7, 'names': array([ 2.,  1.,  7.,  4.,  5.,  6.,  3.])},
 {'length': 7, 'names': array([ 2.,  3.,  7.,  5.,  4.,  6.,  1.])},
 {'length': 3, 'names': array([ 2.,  1.,  3.])},
 {},
 {'length': 2, 'names': array([ 2.,  1.])},
 {'length': 2, 'names': array([ 2.,  1.])}]

In [58]:
final_index_limit= []
for i in range(len(index_limit)):
    x = index_limit[i]
    try:
        final_index_limit.append(x['length'])
    except:
        pass

In [59]:
final_index_limit, sum(final_index_limit)

([21, 2, 4, 4, 3, 4, 3, 5, 7, 7, 7, 3, 2, 2], 74)

## Make the contstraint list

In [60]:
from random import *
randBinList = lambda n: [randint(0,1) for b in range(1,n+1)]
con_index = []
x = randBinList(181)
for index in final_index_limit:
    temp = []
    temp.append(sum(x[:index]) - 1)
    temp = temp*index
    con_index=con_index + temp

In [61]:
#con_index
df_new.columns

Index(['userID', 'itemID', 'age_15.0', 'age_17.0', 'age_18.0', 'age_19.0',
       'age_20.0', 'age_21.0', 'age_22.0', 'age_23.0', 'age_24.0', 'age_25.0',
       'age_26.0', 'age_27.0', 'age_28.0', 'age_29.0', 'age_30.0', 'age_31.0',
       'age_33.0', 'age_34.0', 'age_35.0', 'age_36.0', 'age_63.0', 'sex_1',
       'sex_2', 'country_2', 'country_3', 'country_4', 'country_5', 'time_1.0',
       'time_2.0', 'time_3.0', 'time_4.0', 'daytype_1.0', 'daytype_2.0',
       'daytype_3.0', 'season_1.0', 'season_2.0', 'season_3.0', 'season_4.0',
       'location_1.0', 'location_2.0', 'location_3.0', 'weather_1.0',
       'weather_2.0', 'weather_3.0', 'weather_4.0', 'weather_5.0',
       'social_1.0', 'social_2.0', 'social_3.0', 'social_4.0', 'social_5.0',
       'social_6.0', 'social_7.0', 'endEmo_1.0', 'endEmo_2.0', 'endEmo_3.0',
       'endEmo_4.0', 'endEmo_5.0', 'endEmo_6.0', 'endEmo_7.0',
       'dominantEmo_1.0', 'dominantEmo_2.0', 'dominantEmo_3.0',
       'dominantEmo_4.0', 'dominantEmo_5.0

# Pyswarm Implementation

In [62]:
# from pyswarm import pso
# import time
# from random import *

In [63]:
# start_time = time.time()
# #user_context = df_new.iloc[0][2:-1].tolist()
# user_rating = df_new.rating.tolist()[:20]
# def banana(x):
#     lhs = 0.0
#     square_error = 0.0
#     for i in range(20):
#         user_context = df_new.iloc[i][2:-1].tolist()
#         lhs = np.dot(x,user_context)
# #         for weight, context in zip(x, user_context):
# #             lhs += weight * context
#         square_error += (user_rating[i] - lhs)**2
#     print(square_error)
#     return square_error/2

# def con(x):
#     con_index = []
#     final_index_limit = [21, 2, 4, 4, 3, 4, 3, 5, 7, 7, 7, 3, 2, 2]
#     start_index=0
#     con_index = []
#     for index in final_index_limit:
#         temp = []
#         temp.append(sum(x[start_index:start_index+index]) - 1)
#         temp = temp*index
#         con_index=con_index + temp
#         start_index = index
#     return con_index

# lb = [0]*74
# ub = [1]*74

# xopt, fopt = pso(banana,lb, ub,f_ieqcons=con,swarmsize=740,maxiter=1000)
# print("--- %s seconds ---" % (time.time() - start_time))

In [64]:
# Rating
# fopt

In [65]:
# print("----------------------Difference Between Ratings---------------------")
# for i in range(10):
#     rating = 0
#     count = 0
#     original_rating = df_new.iloc[i][-1]
#     predicted_rating = np.dot(xopt, df_new.iloc[i][2:-1].tolist())
#     print(predicted_rating, original_rating)
#     print(str(float(original_rating - original_rating)))
        

In [66]:
# original_rating = [df_new.iloc[i][-1] for i in range(1000)]
# predicted_rating = [np.dot(xopt, df_new.iloc[i][2:-1].tolist()) for i in range(1000)]
# print("----------------------  RMSE VALUE IS  ---------------------")
# mean_squared_error(original_rating,predicted_rating)

In [67]:
#np.dot(x,y)

In [68]:
# x = [1]*78
# final_index_limit = [21, 2, 4, 4, 3, 4, 3, 5, 7, 7, 7, 3, 2, 2]
# start_index=0
# con_index = []
# for index in final_index_limit:
#     temp = []
#     temp.append(sum(x[start_index:start_index+index]) - 1)
#     temp = temp*index
#     con_index=con_index + temp
#     start_index = index
# return con_index

### Firefly Optimization Implementation

In [69]:
#user_rating = df_new.rating.tolist()[:20]

In [78]:
# from metaheuristic_algorithms.function_wrappers.abstract_wrapper import AbstractWrapper
# class MinimizeWeightsSumSquareError(AbstractWrapper):

#     def maximum_decision_variable_values(self):
#         return [1]*74

#     def minimum_decision_variable_values(self):
#         return [0]*74

#     def objective_function_value(self, decision_variable_values):
#         user_rating = df_new.rating.tolist()[:20]
#         lhs = 0.0
#         square_error = 0.0
#         for i in range(20):
#             user_context = df_new.iloc[i][2:-1].tolist()
#             lhs = np.dot(decision_variable_values,user_context)
#             square_error += (user_rating[i] - lhs)**2
#         return square_error/2

#     def initial_decision_variable_value_estimates(self):
#         return [0.0]*74

In [ ]:
# import time
# from metaheuristic_algorithms.firefly_algorithm import FireflyAlgorithm
# start_time = time.time()
# #from metaheuristic_algorithms.function_wrappers.rosenbrook_function_wrapper import RosenbrookFunctionWrapper
# #from metaheuristic_algorithms.function_wrappers.nonsmooth_multipeak_function_wrapper import NonsmoothMultipeakFunctionWrapper
# minimize_weights_function_wrapper = MinimizeWeightsSumSquareError()

# number_of_variables = 74
# objective = "minimization"

# firefly_algo = FireflyAlgorithm(minimize_weights_function_wrapper, number_of_variables, objective)


# number_of_fireflies = 60
# maximun_generation = 300
# randomization_parameter_alpha = 0.3
# absorption_coefficient_gamma = 1.1

# result = firefly_algo.search(number_of_fireflies = number_of_fireflies, 
#                                maximun_generation = maximun_generation, 
#                                randomization_parameter_alpha = randomization_parameter_alpha, 
#                                absorption_coefficient_gamma = absorption_coefficient_gamma)
# print(result["best_decision_variable_values"])
# print("--- %s seconds ---" % (time.time() - start_time))
# print(result["best_objective_function_value"]) 

In [79]:
# with open("result.txt", "w") as myfile:
#     for key in sorted(result):
#         myfile.write(str(key) + "," + ",".join(str(result[key])) + "\n")
